In [20]:
import options as opt
import numpy as np
from typing import Callable
import pandas as pd

In [21]:
spot = 41.0
strike = 40.0
expiry = 1
vol = 0.3
rate = 0.08
div = 0.0
num = 200

In [22]:
binomial_call = opt.european_binomial_pricer(spot, strike, expiry, rate, div, vol, num, payoff=opt.call_payoff)
binomial_put = opt.european_binomial_pricer(spot, strike, expiry, rate, div, vol, num, payoff=opt.put_payoff)
black_scholes_call = opt.black_scholes_call(spot, strike, expiry, rate, div, vol)
black_scholes_put = opt.black_scholes_put(spot, strike, expiry, rate, div, vol)

In [23]:
def monte_carlo_sim(spot:float, strike:float, expiry: int, vol:float, rate:float, div:float, payoff: Callable, M:int):
    
    # initialize space to save the terminal price at the end of each binomial path
    terminal_prices = np.zeros(M)

    # get M binomial paths
    for i in range(M):
        terminal_prices[i] = opt.binomial_path(spot, expiry, rate, div, vol, num)[-1]

    option_payoffs = payoff(terminal_prices, strike)
    average_payoff = np.mean(option_payoffs)
    standard_error_payoff = np.std(option_payoffs) / np.sqrt(M)

    discount_factor = np.exp(-rate * expiry)
    discounted_mean = average_payoff * discount_factor

    return(discounted_mean, standard_error_payoff)

In [24]:
M = [10000, 25000, 50000, 75000, 100000]
call_mean_premiums = np.zeros(len(M))
call_standard_error = np.zeros(len(M))
put_mean_premiums = np.zeros(len(M))
put_standard_error = np.zeros(len(M))

# call the monte carlo simulation for each number of iterations and store results
for i in range(len(M)):
    
    call_mean_premiums[i], call_standard_error[i] = monte_carlo_sim(spot, strike, expiry, vol, rate, div, opt.call_payoff, M[i])
    put_mean_premiums[i], put_standard_error[i] = monte_carlo_put = monte_carlo_sim(spot, strike, expiry, vol, rate, div, opt.put_payoff, M[i])

# Conclusion

In [25]:
results = pd.DataFrame({'Call Mean': call_mean_premiums, "Call Std Error": call_standard_error, 'Put Mean': put_mean_premiums, "Put Std Error": put_standard_error}, index=M)
print(results)

        Call Mean  Call Std Error  Put Mean  Put Std Error
10000    7.052766        0.107700  2.925108       0.049546
25000    6.951859        0.067981  2.857715       0.031000
50000    6.937542        0.047624  2.895994       0.021979
75000    6.915603        0.038795  2.903451       0.018094
100000   6.972291        0.033717  2.902565       0.015616


In [26]:
print(f"binomial model call price was: {binomial_call} and the put price was {binomial_put}")
print(f"black scholes model call price was: {black_scholes_call} and the put price was {black_scholes_put}")

binomial model call price was: 6.966101170227521 and the put price was 2.890755025692191
black scholes model call price was: 6.960998922548743 and the put price was 2.8856527780141707


The monte carlo simulations provide a very close approximate to the answers provided by the binomial and Black-Scholes models